### 迭代器

以直接作用于for循环的数据类型有以下几种：

一类是集合数据类型（list、tuple、dict、set、str）,一类是generator包括生成器和yield，这些可以直接作用于for循环的对象都统称为可迭代对象。

#### 判断是否可迭代

可用isinstance()判断一个对象是否是 Iterable对象：

In [1]:
from collections import Iterable
isinstance("abc",Iterable)  #True,可迭代
isinstance(100,Iterable)    #False,不可迭代

False

#### 可以被next()函数调用并不断返回下一个值的对象称为迭代器：Iterator.可以使用isinstance()判断一个对象是否是Iterator对象：

In [5]:
from collections import Iterator
isinstance((x for x in range(10)),Iterator)

True

In [6]:
from collections import Iterator
isinstance([],Iterator)

False

#### 生成器都是Iterator,但list、dict/str虽然是Iteratable,却不是Iterator。把list、dict、str等Iterable可以使用iter()函数

In [8]:
isinstance(iter([]),Iterator)

True

In [9]:
isinstance(iter("abc"),Iterator)

True

#### 总结：凡是可作用于for循环的对象都是 Iterable类型；
   ####     凡是作用于next()的函数的对象都是Iterator类型；
####            集合数据类型如list、dict、str等是Iterable 但不是 Iterator，不过可以通过iter()函数获得一个Iterator对象。

### 闭包

在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，那么我们将这个函数以及用到的一些变量，里面的整体为闭包

In [10]:
#定义一个函数
def test(number):
    
    #在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，那么我们将这个函数以及用到的一些变量，里面的整体为闭包
    def test_in(number_in):
        print("in test_in 函数，number_in is %d" % number_in)
        return number+number_in
    
    #其实这里返回的就是闭包的结果
    return test_in                  #####return了 test_in 指向的函数体

#给test函数赋值，这个20就是给参数number
ret = test(20)

#注意这里的100其实给参数number_in
print(ret(100))

#注意这里的200其实给参数number_in\
print(ret(200))

in test_in 函数，number_in is 100
120
in test_in 函数，number_in is 200
220


In [11]:
#闭包特点:当执行test(100)返回number为100，再次执行ret()会使用上一次的number值最后得到结果101
def test(number):
    print("---1---")
    def test_in(number2):
        print("---2---")
        print(number+number2)
    print("---3---")
    return test_in   #返回test_in指向test_in()函数的引用

ret = test(100) #ret指向test_in，也就是全部指向test_in()的函数体
print("-"*10)
ret(1)

---1---
---3---
----------
---2---
101


#### 闭包的应用

1、简化调用方式，比传统写法调用简便（a,b不变，只变化x）

In [12]:
def line_conf(a, b):
    
    def line(x):
        return a*x + b
    
    return line

line1 = line_conf(1,2)
line2 = line_conf(4,5)
print(line1(5))
print(line2(5))

7
25


### 装饰器

在不改变原函数功能的情况下添加新功能

In [16]:
def w1(func):
    def inner():
        print("- - -正在验证权限- - -")
        if True:
            func()
        else:
            print("没有权限")
    return inner

@w1    
def f1():
    print("- - -f1- - -")

@w1
def f2():
    print("- - -f2- - -")
    
# innerFunc = w1(f1)
# innerFunc()

#f1 = w1(f1)  # 等价于 @w1
f1()
f2()

- - -正在验证权限- - -
- - -f1- - -
- - -正在验证权限- - -
- - -f2- - -


### 2个装饰器

In [4]:
#定义函数：完成包裹数据
def makeBold(fn):
    def wrapped():
        print("----1---")
        return "<b>" + fn() + "</b>"
    return wrapped

#定义函数：完成包裹数据
def makeItalic(fn):
    def wrapped():
        print("----2---")
        return "<i>" + fn() + "</i>"
    return wrapped

#先执行装饰器@makeItalic
@makeBold     #test3 = makeBold(test3)
@makeItalic   #test3 = makeItalic(test3)
def test3():
    print("----3---")
    return "hello world-3"

ret = test3()
print(ret)

#执行流程:
#1.@makeBold先执行，打印 print("----1---")，return wrapped函数，fn()传入调用@makeItalic
#2.@makeItalic继续执行，打印 print("----2---")，return wrapped函数，fn()传入调用test3()，输出 print("----3---")和<i>hello world-3</i>
#3.最后@makeBold才调用@makeItalic的return值

----1---
----2---
----3---
<b><i>hello world-3</i></b>


### 装饰器执行的时间

In [6]:
def w1(func):
    print("---正在装饰1----")
    def inner():
        print("---正在验证权限1----")
        func()
    return inner

def w2(func):
    print("---正在装饰2----")
    def inner():
        print("---正在验证权限2----")
        func()
    return inner

#只要python解释器执行到了这个代码,那么就会自动的进行装饰,而不是等到调用的时候才装饰的
@w1
@w2
def f1():
    print("---f1---")

#在调用f1之前,已经进行装饰了
f1()

---正在装饰2----
---正在装饰1----
---正在验证权限1----
---正在验证权限2----
---f1---


### 装饰器对有参数函数进行装饰

In [4]:
#不带参数的装饰器
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in():
        print("- - -func_in - - 1 - -")
        functionName()
        print("- - -func_in - - 2 - -")
    print("- - -func- - - 2 - -")
    return func_in

@func
def test():
    print("- - -test- - -")
    
# test = func(test)
test()

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test- - -
- - -func_in - - 2 - -


In [9]:
#定长参数
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in(a,b):  #如果a,b 没有定义,那么会导致16行的调用失败
        print("- - -func_in - - 1 - -")
        functionName(a,b)#如果没有把a,b当做实参进行传递,那么会导致调用12行的函数失败
        print("- - -func_in - - 2 - -")
    print("- - -func- - - 2 - -")
    return func_in

@func
def test(a,b):
    print("- - -test-a=%d,b=%d- - -"%(a,b))

test(11,22)

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test-a=11,b=22- - -
- - -func_in - - 2 - -


In [12]:
#不定长参数
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in(*args,**kwargs):#采用不定长参数的方式满足所有函数需要参数以及不需要参数的情况
        print("- - -func_in - - 1 - -")
        functionName(*args,**kwargs)#这个地方,需要写*以及**,如果不写的话,那么args是元祖,而kwargs是字典
        print("- - -func_in - - 2 - -")
    print("- - -func- - - 2 - -")
    return func_in

@func
def test(a,b,c):
    print("- - -test-a=%d,b=%d,c=%d- - -"%(a,b,c))
    
@func
def test2(a,b,c,d):
    print("- - -test-a=%d,b=%d,c=%d,d=%d- - -"%(a,b,c,d))
    
test(11,22,33)
test2(44,55,66,77)

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test-a=11,b=22,c=33- - -
- - -func_in - - 2 - -
- - -func_in - - 1 - -
- - -test-a=44,b=55,c=66,d=77- - -
- - -func_in - - 2 - -


### 装饰器对带有返回值的函数进行装饰

In [14]:
#返回值为空
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in():
        print("- - -func_in - - 1 - -")
        functionName()
        print("- - -func_in - - 2 - -")
    print("- - -func- - - 2 - -")
    return func_in

@func 
def test():
    print("- - -test- - -")
    return "Hello world"

ret = test()
print("test return value is %s"%ret)

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test- - -
- - -func_in - - 2 - -
test return value is None


In [21]:
#有返回值
def func(functionName):
    print("- - -func- - - 1 - -")
    def func_in():
        print("- - -func_in - - 1 - -")
        ret = functionName() #保存返回来的 Hello world
        print("- - -func_in - - 2 - -")
        return ret  #把 Helloworld 返回到 ret=test()处的调用
    print("- - -func- - - 2 - -")
    return func_in

@func 
def test():
    print("- - -test- - -")
    return "Hello world"

ret = test()
print("test return value is %s"%ret)
### ret = test() 中的 test()函数指向func_in(),去调func_in()函数，当func_in()函数无返回值时，test()函数就没有返回值，即为None
### functionName指向 def test()这个函数，故执行完functionName()后，functionName得到了返回值 Hello world，此时找个变量接收functionName()
#   执行后的返回值("Hello world"),在return，就可以让func_in()函数有返回值，即ret=test()中的test()执行后有返回值

- - -func- - - 1 - -
- - -func- - - 2 - -
- - -func_in - - 1 - -
- - -test- - -
- - -func_in - - 2 - -
test return value is Hello world


### 通用装饰器

In [20]:
def func(functionName):
    def func_in(*args,**kwargs):
        print("- - -记录日志- - -")
        ret = functionName(*args,**kwargs)
        return ret
    
    return func_in

@func 
def test():
    print("- - -test- - -")
    return "Hello world"

@func
def test2():
    print("- - -test2- - -")
    
@func
def test3(a):
    print("- - - test3-a=%d - - -"%a)

ret = test()
print("test return value is %s"%ret)

test2()

test3(11)

- - -记录日志- - -
- - -test- - -
test return value is Hello world
- - -记录日志- - -
- - -test2- - -
- - -记录日志- - -
- - - test3-a=11 - - -


### 带有参数的装饰器

In [13]:
def func_arg(arg):
    def func(functionName):
        def func_in():
            print("- - -记录日志- arg=%s- -"%arg) 
            if arg == "heihei": #根据传入的参数不同，装饰的内容不同
                functionName()
                functionName()
            else:
                functionName()
        return func_in
    return func

#1. 先执行func_arg("heihei")函数,这个函数return 的结果是func这个函数的引用
#2. @func
#3. 使用@func对test进行装饰
@func_arg("heihei")
def test():
    print("- - -test- - -")
    
test()

#带有参数的装饰器,能够起到在运行时,有不同的功能
@func_arg("haha")
def test2():
    print("--test2--")
    
test2()

- - -记录日志- arg=heihei- -
- - -test- - -
- - -test- - -
- - -记录日志- arg=haha- -
--test2--


### 作用域

globals()查看全局变量和locals()查看局部变量

LEGB规则：Python使用LEGB的顺序来查找一个符号对应的对象

#### locals->enclosing function -> globals -> builtins

#### locals:当前所在命名空间（如函数、模块），函数的参数也属于命名空间内的变量

#### enclosing:外部嵌套函数的命名空间（闭包中常见）

In [17]:
def fun1():
    a = 10
    def fun2():
        # a位于外部嵌套函数的命名空间
        print(a)
    return fun2

ret = fun1()
ret()

10


#### globals:全局变量，函数定义所在模块的命名空间

In [19]:
a = 1
def fun():
    #需要通过 global 指令来声明全局变量
    global a 
    #修改全局变了
    a = 2
    print(a)
    
fun()

2


### Python动态添加属性以及方法

#### 给对象添加属性

In [26]:
class Person(object):
    def __init__(self, newName, newAge):
        self.name = newName
        self.age = newAge
        
laowang = Person("老王",30)
print(laowang.name)
print(laowang.age)

laowang.addr = "北京..."
print(laowang.addr)

laozhao = Person("老赵",25)
# print(laozhao.addr)   # 报错，老赵没有addr属性

#添加类属性
Person.num = 100
print(laowang.num)
print(laozhao.num)

老王
30
北京...
100
100


#### 给对象添加方法

In [27]:
import types

class Person(object):
    def __init__(self, newName, newAge):
        self.name = newName
        self.age = newAge
    
    def eat(self):
        print("- - -%s正在吃- - -"%self.name)


def run(self):
    print("- - -%s正在跑- - -"%self.name)


p1 = Person("p1", 10)
p1.eat()
#添加方法,types.MethodType(函数名,对象名)
# p1.run = run
# p1.run()   # 虽然p1对象中 run 属性已经指向了def run()函数，但是这句代码还不正确，因为
             #run属性指向的函数，是后来添加的，即p1.run() 的时候，并没把p1当作第一个参数，
            # 导致了第10行的函数调用的时候，出现缺少参数的问题
p1.run = types.MethodType(run,p1)
p1.run()

- - -p1正在吃- - -
- - -p1正在跑- - -


#### 给类添加静态属性

In [33]:
class P(object):
    def __init__(self, newName):
        self.name = newName

@staticmethod
def test():
    print("- - -static method- - -")
    
P.test = test
P.test()
# P.xxx = test  #一般用上面一种添加、调用方式
# P.xxx()

- - -static method- - -
- - -static method- - -


#### 给类添加类方法

In [34]:
class P(object):
    def __init__(self, newName):
        self.name = newName
        
@classmethod
def printNum(cls):
    print("- - - class method - - -")
    
P.printNum = printNum
P.printNum()

- - - class method - - -


### \__slots\__的作用

#### 为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的\__slots\__变量，来限制该class实例能添加的属性：

#### 使用\__slots\__要注意，\__slots\__定义的属性仅对当前类实例起作用，对继承的子类是不起作用的

In [38]:
class Person(object):
    __slots__ = ("name","age")
    
P = Person()
P.name = "老王"
P.age = 20
P.score = 100  #不允许添加该属性， __slots__限制了只能添加 name、age 属性

AttributeError: 'Person' object has no attribute 'score'

### 生成器

#### 通过列表生成式，可以直接创建一个列表。在Python中当列表生成式计算量太大，一边循环一边计算的机制，称为生成器：generator

#### 创建生成器方法1--把列表生成式的中括号改成小括号

In [40]:
# 创建生成器方法1--把列表生成式的中括号改成小括号
b = (x*2 for x in range(10))
print(b)

# next() 逐个生成,每执行一次生成一个，到最后一个还去生成会产生异常
print(next(b))
print(next(b))
#next(b)
#next(b)

<generator object <genexpr> at 0x0000024897661938>
0
2


#### 创建生成器方法2--当函数中有yield则此函数为生成器

In [46]:
#斐波那契数列，1 1 2 3 5 8 13 21 34
def creatnum():
    a,b = 0,1
    for i in range(5):
        print("- -1- -")
        yield b       #当 运行到yield,下面代码不运行，并保存b并返回
        print("- -2- -")
        a,b = b,a+b
        print("- - -3- - -")
    print("- - -stop- - -")

#创建一个生成器对象
a = creatnum()
print(next(a))  #再次执行，会返回2 ，到最后一个还去生成会产生异常
print(next(a))  #再次执行next(a)，yield b下面的代码开始执行，当从上到下再次到yield时，又停止执行

- -1- -
1
- -2- -
- - -3- - -
- -1- -
1


### 生成器2--send

In [48]:
def creatnum():
    a,b = 0,1
    for i in range(5):
        print("- -1- -")
        yield b       #当 运行到yield,下面代码不运行，并保存b并返回
        print("- -2- -")
        a,b = b,a+b
        print("- - -3- - -")
    print("- - -stop- - -")

#创建一个生成器对象
a = creatnum()

for num in a: #用for循环执行完不会产生异常
    print(num)  

- -1- -
1
- -2- -
- - -3- - -
- -1- -
1
- -2- -
- - -3- - -
- -1- -
2
- -2- -
- - -3- - -
- -1- -
3
- -2- -
- - -3- - -
- -1- -
5
- -2- -
- - -3- - -
- - -stop- - -


#### \__next\__()

In [50]:
def creatnum():
    a,b = 0,1
    for i in range(5):
        print("- - -1- - -")
        yield b       #当运行到yield,后面不运行，并保存b并返回
        print("- - -2- - -")
        a,b = b,a+b

a = creatnum()
ret = a.__next__()   #这个方式和next(a)等价
print(ret)

- - -1- - -
1


#### send

In [54]:
def test():
    i = 1
    while i < 5:
        temp = yield i  #temp在没传值的情况下，不是i值，而是None空值 ## yield后面是什么，就返回什么
        print(temp)
        i +=1
t = test()
print(t.__next__())  #再次执行会返回下一个值
print(next(t))
print(next(t))
print(next(t))

1
None
2
None
3
None
4


In [57]:
def test():
    i = 1
    while i < 5:
        temp = yield i  #temp在没传值的情况下，不是i值，而是None空值 ## yield后面是什么，就返回什么
        print(temp)
        i +=1
t = test()
print(next(t))
t.send("hh")  #当执行这一句会执行temp = yield i，给temp传值hh
              # .send()、__next__()、next()，都可以让生成器往下走，只是send()会传值给 yield i 整体
              # .send()不能用作第一次执行生成器(第一次执行停在yield处，无法传值)，用next()、__next__()先执行一次生成器
              # 或者用send()第一次调用生成器，这样写 send(None)

1
hh


2

上面第一次执行生成器时，停在了temp=yield i,并没有执行print(temp)，所以第一次执行，只是yield返回的1，没有打印temp

### 生成器4--多任务

#### 多任务中的一种--协程（还有进程、线程）

In [58]:
def test1():
    while True:
        print("- - -1- - -")
        yield None
        
def test2():
    while True:
        print("- - -2- - -")
        yield None
        
t1 = test1()
t2 = test2()
i = 0
while i < 3:
    t1.__next__()#此处到 打印 - -1- -到yield停住，执行下一个 t2.__next__()
    t2.__next__()#此处到 打印 - -2- -到yield停住，执行下面的代码
    i +=1
#三个while一直循环执行，看上去同时执行的东西就是多任务（会造成同时执行的假象，其实不是）

- - -1- - -
- - -2- - -
- - -1- - -
- - -2- - -
- - -1- - -
- - -2- - -
